In [1]:
import networkx as nx;
import matplotlib.pyplot as plt
import pandas as pd;
import gurobipy as gp;
from gurobipy import GRB;
import csv;
import sys;
import time;
from datetime import datetime;
import math;

In [2]:
#'''
# Creating Results Files ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

file_summary = open("Results_Summary.csv", "w");

file_summary.write('Instance\t, Budget\t, calA_Level\t, Nodes\t, Arcs\t, Opt_Gap\t, Obj_value\t,');
file_summary.write('Flow\t, Level_1\t, Level_2\t, Level_3\t, Other_Levels\t, Run_Time\n');
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#'''



num = 2;

m = {};
budget = [0, 5, 10, 15];
rate = [1];
network = {};

T_Limit = 10;


for n in range(num):
    m[n] = {};
    network[n] = 'Net'+str(n+1);
    
    
    for b in range(len(budget)):
        m[n][b] = {};       
        
        for r in range(len(rate)):
            
            print("\n");
            print(network[n], ', budget', budget[b]);
            
            start_time = time.time();
            
            with open(network[n]+'.csv', newline='') as f:
                reader = csv.reader(f);
                row1 = next(reader);
                s = int(row1[0]);
                t = int(row1[1]);
                calA_level = int(row1[2]);
    
                G = nx.DiGraph();
                data = pd.read_csv(network[n]+'.csv',skiprows=1, header=None);
                n_edge = len(data.index+1);
        
                for i in range(n_edge): 
                    G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], level=data.iat[i,5]);
                    

                    
                A = 0;
                for i,j in G.edges:
                    if G.edges[i,j]['special'] == 1:
                        A = A + 1;
                  
            
            m[n][b][r] = gp.Model('GenFormulation'+str(n)+str(b)+str(r));
            m[n][b][r].setParam('TimeLimit', T_Limit);
            
            # Variables

            z = m[n][b][r].addVars(G.edges, vtype=GRB.BINARY);

            gamma = m[n][b][r].addVars(G.edges, vtype=GRB.BINARY);

            theta = m[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);

            beta = m[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);

            x = m[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);

            alpha = m[n][b][r].addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);

            
            # Constraints

            m[n][b][r].addConstr(gp.quicksum(G.edges[i,j]['cost']*gamma[i,j] for i,j in G.edges) <= budget[b]);

            m[n][b][r].addConstr(gp.quicksum(G.edges[i,j]['capacity']*beta[i,j] for i,j in G.edges) <= x[t,s]);

            m[n][b][r].addConstr(alpha[t] - alpha[s] >= 1);

            for i,j in G.edges:
                if G.edges[i,j]['special'] == 1:
                    m[n][b][r].addConstr(x[i,j] - G.edges[i,j]['capacity']*z[i,j] <= 0);
    
                m[n][b][r].addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
    
                m[n][b][r].addConstr(beta[i,j] + gamma[i,j] - theta[i,j] >= 0);
    
                m[n][b][r].addConstr(x[i,j] - G.edges[i,j]['capacity']*(1-gamma[i,j]) <= 0);
    
            m[n][b][r].addConstrs(gp.quicksum(x[v,u] for u in G.successors(v)) 
                                  - gp.quicksum(x[u,v] for u in G.predecessors(v)) == 0 for v in G.nodes);
            
                # Objective
            
            m[n][b][r].setObjective(gp.quicksum(G.edges[i,j]['special']*z[i,j] for i,j in G.edges), GRB.MINIMIZE);
            
            m[n][b][r].setParam("IntegralityFocus",1);
            m[n][b][r].setParam("NumericFocus",2);
            #m[n][b][r].setParam("IntFeasTol",1e-9)
            
            
            m[n][b][r].update();
            m[n][b][r].setParam("OutputFlag", 0);
            m[n][b][r].optimize();

            
# ~~~~~~~~~~~~~~~ Printing Results ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            
            file = open('results_'+network[n]+'_b'+str(budget[b])+'.txt', "w");
            
            file.write('Number of Nodes: %g' % (t+1) +'\n');
            file.write('Number of Arcs: %g' % G.number_of_edges() +'\n');
            file.write('Budget: %g' % budget[b] +'\n\n');
            
            
            
            if m[n][b][r].status == GRB.status.OPTIMAL:
                for i,j in G.edges:
                    if gamma[i,j].x > 0.0000001:
                        print('Interdict: ',i,j);
                               
                obj = m[n][b][r].objVal;
                OptGap = 0.0;
                sol = A - obj;
                file.write('Solved to Optimality \n');
                file.write('Number of special arcs without flow: %g' % sol +'\n');
                file.write('Number of special arcs with flow: %g' % obj +'\n');
                
            elif m[n][b][r].status == GRB.status.TIME_LIMIT:
                for i,j in G.edges:
                    if gamma[i,j].x > 0.0000001:
                        print('Interdict: ',i,j);
                
                UB = m[n][b][r].objVal;
                LB = m[n][b][r].objBound;
                sol = A - UB;
                file.write('Reached Time Limit \n');
                file.write('Number of special arcs without flow: %g' % sol +'\n');
                file.write('Number of special arcs with flow: %g' % UB +'\n');
                file.write('UB (Sub): %g' % UB +'\n'); 
                file.write('LB (MP): %g' % LB +'\n');
                OptGap = (UB-LB)/UB;
                file.write('Opt Gap: %s' % str("{:.2f}".format((OptGap)))+'\n');
            
            
            file.write('Max-Flow: %g' % x[t, s].x +'\n');
            file.write('Run-time: %s' % "{:.2f}".format((time.time() - start_time)) + ' sec');
            file.write('\n\n');    
            file.write('Level 0 Capacities: ' + str(G.edges[s,1]['capacity']) +'\n\n');
            
            
            lev1 = 0;
            lev2 = 0;
            lev3 = 0;
            other_level = 0;
            file.write('\nInterdiction plan: \n');
            
            key1 = False;
            key2 = False;
            key3 = False;
                
            for i, j in G.edges: 
                if gamma[i,j].x > 0.0000001:
                    if G.edges[i,j]["level"] == 1:
                        if key1 == False:
                            file.write("Level 1:" +'\n');
                            key1 = True;
                        file.write('Arc (%s,%s)' %(i,j) +'\n');
                        lev1 += 1;
                    elif G.edges[i,j]["level"] == 2:
                        if key2 == False:
                            file.write('Level 2:' +'\n');
                            key2 = True;
                        file.write('Arc (%s,%s)' %(i,j) +'\n');
                        lev2 += 1;
                    elif G.edges[i,j]["level"] == 3:
                        if key3 == False:
                            file.write('Level 3:' +'\n');
                            key3 = True;
                        file.write('Arc (%s,%s)' %(i,j) +'\n');
                        lev3 += 1;
                    else:
                        file.write('Other Levels:' +'\n');
                        file.write('Arc (%s,%s)' %(i,j) +'\n');
                        other_level += 1;
                   
                        
            
            file.write('\n');
            file.write('Special arcs with flow' +'\n');    
                
            for i, j in G.edges:
                if G.edges[i,j]['special'] == 1:
                    if x[i,j].x > 0.0000001:
                        file.write('Arc (%s,%s): ' %(i,j) + '\t Flow: %f' % x[i,j].x +'\n');
                        
                        
            file_summary.write(network[n]+'_b'+str(budget[b])+','+str(budget[b])+','+str(calA_level)+','
                               +str(t+1)+','+str(G.number_of_edges())+','+str("{:.3f}".format((OptGap)))+','+str(obj)+','
                               +str("{:.2f}".format(x[t, s].x))+','+str(lev1)+','+str(lev2)+','+str(lev3)+','
                               +str(other_level)+','+str("{:.2f}".format((time.time() - start_time)))+'sec\n');
            
            
            file.write('\n');
            file.close();
                         
file_summary.close();



Net1 , budget 0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-30
Set parameter TimeLimit to value 10
Set parameter IntegralityFocus to value 1
Set parameter NumericFocus to value 2


Net1 , budget 5
Set parameter TimeLimit to value 10
Set parameter IntegralityFocus to value 1
Set parameter NumericFocus to value 2
Interdict:  11 23
Interdict:  11 24
Interdict:  11 27
Interdict:  11 28
Interdict:  13 28


Net1 , budget 10
Set parameter TimeLimit to value 10
Set parameter IntegralityFocus to value 1
Set parameter NumericFocus to value 2
Interdict:  11 23
Interdict:  11 26
Interdict:  11 27
Interdict:  11 28
Interdict:  11 30
Interdict:  18 25
Interdict:  18 29
Interdict:  18 30
Interdict:  13 25
Interdict:  13 28


Net1 , budget 15
Set parameter TimeLimit to value 10
Set parameter IntegralityFocus to value 1
Set parameter NumericFocus to value 2
Interdict:  6 19
Interdict:  11 23
Interdict:  11 24
Interdict:  11 26
Interdict:  11 27
Interdict:  